In [169]:
# importing the necessary libraries and loading an image using OpenCV
from functions import *
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import morphology
from sklearn.mixture import GaussianMixture
from scipy import ndimage
import os
from scipy.stats import skew, kurtosis
from skimage.feature import graycomatrix, graycoprops
from skimage.feature import local_binary_pattern as LBP
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import cohen_kappa_score as kappa
from imblearn.under_sampling import RandomUnderSampler as RUS
from imblearn.ensemble import BalancedBaggingClassifier as BBC
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE

In [ ]:
# Define a folder to save the trained segmented images
output_folder = "tc_train_segmented_images"
os.makedirs(output_folder, exist_ok=True)

# Load from training data
# Load images and assign labels
mel_images, mel_labels = load_images_from_folder_with_labels("threeclass/train/mel", 0)
bcc_images, bcc_labels = load_images_from_folder_with_labels("threeclass/train/bcc", 1)
scc_images, scc_labels = load_images_from_folder_with_labels("threeclass/train/scc", 2)

train_images_tc = mel_images + bcc_images + scc_images
train_labels_tc = mel_labels + bcc_labels + scc_labels

In [ ]:
# CROPPING OF THE TRAINING IMAGES
# Lists to store segmented images and features
segmented_images = []
tc_train_features_list2 = []

for i, image in enumerate(train_images_tc):
    print("Loading image... ", i)
    orig_image = image #save the original image for later
    
    # Firstly we detect if there is an halo and crop the image in that case
    preimg = image[:,:,0] #we need only one channel
    pdim = np.shape(preimg)
    
    # These are the corner pixels
    lu = [0,0]
    ru = [0, pdim[1]-1]
    ld = [pdim[0]-1, 0]
    rd = [pdim[0]-1, pdim[1]-1]

    thr = 40 # Intensity threshold under which is considered to be halo
    marg = 30 # Extra cropping margin
    
    #We execute the loop until the four corners exceed the intensity threshold. If they are below, we
    #assume we are still in the halo and keeps iterating
    if ((preimg[0,0] == 0) or (preimg[0, pdim[1]-1]==0)
       or (preimg[pdim[0]-1, 0]==0) or (preimg[pdim[0]-1, pdim[1]-1]==0)):
        while(((preimg[lu[0], lu[1]] < thr) or (preimg[ru[0], ru[1]] < thr)
             or (preimg[ld[0], ld[1]] < thr) or (preimg[rd[0], rd[1]] < thr))
             and ((ru[1]>(2*marg)) and (ld[0]>(2*marg)))): #if iterates too much that exceeds img limits, stops
            lu = [lu[0]+1, lu[1]+1]
            ru = [ru[0]+1, ru[1]-1]
            ld = [ld[0]-1, ld[1]+1]
            rd = [rd[0]-1, rd[1]-1]

        image = image[lu[0]+marg:ld[0]-marg, ru[0]+marg:rd[0]-marg, :]
    
        if ((np.shape(image)[0] < marg) or (np.shape(image)[1]<marg)): #if the cropping is too much, does a more flexible cropping
            image = orig_image[int(0.3*pdim[0]):int(0.7*pdim[1]), int(0.3*pdim[1]):int(0.7*pdim[1])]        
    
    output_filename = os.path.join(output_folder, f"segmented_{i}.png")
    segmented_region = image
    cv2.imwrite(output_filename, segmented_region)
    # Extract features using the extract_features function
    features = extract_features(segmented_region)
    tc_train_features_list2.append(features)

In [ ]:
# Define a folder to save the validation segmented images
output_folder_2 = "tc_val_segmented_images"
os.makedirs(output_folder_2, exist_ok=True)

# Load from training data
# Load images and assign labels
mel_images, mel_labels = load_images_from_folder_with_labels("threeclass/val/mel", 0)
bcc_images, bcc_labels = load_images_from_folder_with_labels("threeclass/val/bcc", 1)
scc_images, scc_labels = load_images_from_folder_with_labels("threeclass/val/scc", 2)

val_images_tc = mel_images + bcc_images + scc_images
val_labels_tc = mel_labels + bcc_labels + scc_labels

In [ ]:
# CROPPING OF THE VALIDATION IMAGES
# Lists to store segmented images and features
target_size = (227, 227)

output_folder2 = "tc_val_segmented_images"
os.makedirs(output_folder2, exist_ok=True)
target_size = (227, 227)

# Lists to store segmented images and features
segmented_images = []
tc_val_features_list2 = []

for i, image in enumerate(val_images_tc):
    
    print("Loading image... ", i)
    
    orig_image = image #save the original image for later
    
    preimg = image[:,:,0]
    pdim = np.shape(preimg)

    lu = [0,0]
    ru = [0, pdim[1]-1]
    ld = [pdim[0]-1, 0]
    rd = [pdim[0]-1, pdim[1]-1]

    thr = 40
    marg = 30
    
    if ((preimg[0,0] == 0) or (preimg[0, pdim[1]-1]==0)
       or (preimg[pdim[0]-1, 0]==0) or (preimg[pdim[0]-1, pdim[1]-1]==0)):
        while(((preimg[lu[0], lu[1]] < thr) or (preimg[ru[0], ru[1]] < thr)
             or (preimg[ld[0], ld[1]] < thr) or (preimg[rd[0], rd[1]] < thr))
             and ((ru[1]>(2*marg)) and (ld[0]>(2*marg)))): #if iterates too much that exceeds img limits, stops
            lu = [lu[0]+1, lu[1]+1]
            ru = [ru[0]+1, ru[1]-1]
            ld = [ld[0]-1, ld[1]+1]
            rd = [rd[0]-1, rd[1]-1]

        image = image[lu[0]+marg:ld[0]-marg, ru[0]+marg:rd[0]-marg, :]
        
        if ((np.shape(image)[0] < marg) or (np.shape(image)[1]<marg)):
            image = orig_image[int(0.3*pdim[0]):int(0.7*pdim[1]), int(0.3*pdim[1]):int(0.7*pdim[1])]
    
    output_filename = os.path.join(output_folder2, f"segmented_{i}.png")
    segmented_region = image
    cv2.imwrite(output_filename, segmented_region)
    features = extract_features(segmented_region)
    tc_val_features_list2.append(features)

# Now 'segmented_images' contains segmented versions of all your validating images

In [200]:
# FEATURE EXTRACTION OF THE TRAIN:
tc_trainfeats2 = []
P = 4 #bits of the LBP

for i in range(len(train_labels_tc)):
    print("Extracting feature number", i)
    featim = []
    im = cv2.imread('tc_train_segmented_images/segmented_'+str(i)+'.png')
    #Convert the image to different spaces
    imHSV = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    imLab = cv2.cvtColor(im, cv2.COLOR_BGR2Lab)
    imXYZ = cv2.cvtColor(im, cv2.COLOR_BGR2XYZ)
    
    # COLOR FEATURES:
    for nch in range(3):
        #Extracts each of the channels and removes the zero ones (out of the mask)
        im_ch = im[:,:,nch]
        im_ch = im_ch[im_ch!=0]
        im_ch2 = imHSV[:,:,nch]
        im_ch2 = im_ch2[im_ch2!=0]
        im_ch3 = imLab[:,:,nch]
        im_ch3 = im_ch3[im_ch3!=0]
        im_ch4 = imXYZ[:,:,nch]
        im_ch4 = im_ch4[im_ch4!=0]
        
        #For each channel, computes the mean, std, skew and kurosis
        featim.append(np.mean(im_ch))
        featim.append(np.std(im_ch))
        featim.append(skew(np.ravel(im_ch)))
        featim.append(kurtosis(np.ravel(im_ch)))
        
        featim.append(np.mean(im_ch2))
        featim.append(np.std(im_ch2))
        featim.append(skew(np.ravel(im_ch2)))
        featim.append(kurtosis(np.ravel(im_ch2)))
        
        featim.append(np.mean(im_ch3))
        featim.append(np.std(im_ch3))
        featim.append(skew(np.ravel(im_ch3)))
        featim.append(kurtosis(np.ravel(im_ch3))) 
        
        featim.append(np.mean(im_ch4))
        featim.append(np.std(im_ch4))
        featim.append(skew(np.ravel(im_ch4)))
        featim.append(kurtosis(np.ravel(im_ch4))) 
    
    #TEXTURE FEATURES 
    im_gr = np.array(np.mean(im, axis=2), dtype=np.uint8) #gray image
    glcm = graycomatrix(im_gr, distances = [3] , angles = [0]) #computes the GLCM
    featim.append(graycoprops(glcm, 'contrast')[0,0]) #Computes the features related to GLCM
    featim.append(graycoprops(glcm, 'dissimilarity')[0,0])
    featim.append(graycoprops(glcm, 'homogeneity')[0,0])
    featim.append(graycoprops(glcm, 'energy')[0,0])
    featim.append(graycoprops(glcm, 'correlation')[0,0])
    
    #Computes the LBP
    imLBP = LBP(np.mean(im, axis=2, dtype=np.uint8), P, 2)
    histo_im, _ = np.histogram(imLBP, bins=2**P-1)
    histo_im = histo_im[:-1]
    featim = featim + list(histo_im) #each bin of the histogram is one feature
    featim.append(np.mean(histo_im)) #Adds also the mean and standard deviation of the histogram
    featim.append(np.std(histo_im))
        
    tc_trainfeats2.append(featim)

Extracting feature number 0
Extracting feature number 1


C:\Users\arnau\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\arnau\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\arnau\anaconda3\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\arnau\anaconda3\Lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
C:\Users\arnau\anaconda3\Lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\arnau\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1193: RuntimeWarning: Mean of empty slice.
  mean = a.mean(axis, keepdims=Tru

Extracting feature number 2
Extracting feature number 3
Extracting feature number 4
Extracting feature number 5
Extracting feature number 6
Extracting feature number 7
Extracting feature number 8
Extracting feature number 9
Extracting feature number 10
Extracting feature number 11
Extracting feature number 12
Extracting feature number 13
Extracting feature number 14
Extracting feature number 15
Extracting feature number 16
Extracting feature number 17
Extracting feature number 18
Extracting feature number 19
Extracting feature number 20
Extracting feature number 21
Extracting feature number 22
Extracting feature number 23
Extracting feature number 24
Extracting feature number 25
Extracting feature number 26
Extracting feature number 27
Extracting feature number 28
Extracting feature number 29
Extracting feature number 30
Extracting feature number 31
Extracting feature number 32
Extracting feature number 33
Extracting feature number 34
Extracting feature number 35
Extracting feature num

Extracting feature number 279
Extracting feature number 280
Extracting feature number 281
Extracting feature number 282
Extracting feature number 283
Extracting feature number 284
Extracting feature number 285
Extracting feature number 286
Extracting feature number 287
Extracting feature number 288
Extracting feature number 289
Extracting feature number 290
Extracting feature number 291
Extracting feature number 292
Extracting feature number 293
Extracting feature number 294
Extracting feature number 295
Extracting feature number 296
Extracting feature number 297
Extracting feature number 298
Extracting feature number 299
Extracting feature number 300
Extracting feature number 301
Extracting feature number 302
Extracting feature number 303
Extracting feature number 304
Extracting feature number 305
Extracting feature number 306
Extracting feature number 307
Extracting feature number 308
Extracting feature number 309
Extracting feature number 310
Extracting feature number 311
Extracting

Extracting feature number 553
Extracting feature number 554
Extracting feature number 555
Extracting feature number 556
Extracting feature number 557
Extracting feature number 558
Extracting feature number 559
Extracting feature number 560
Extracting feature number 561
Extracting feature number 562
Extracting feature number 563
Extracting feature number 564
Extracting feature number 565
Extracting feature number 566
Extracting feature number 567
Extracting feature number 568
Extracting feature number 569
Extracting feature number 570
Extracting feature number 571
Extracting feature number 572
Extracting feature number 573
Extracting feature number 574
Extracting feature number 575
Extracting feature number 576
Extracting feature number 577
Extracting feature number 578
Extracting feature number 579
Extracting feature number 580
Extracting feature number 581
Extracting feature number 582
Extracting feature number 583
Extracting feature number 584
Extracting feature number 585
Extracting

...

Extracting feature number 4277
Extracting feature number 4278
Extracting feature number 4279
Extracting feature number 4280
Extracting feature number 4281
Extracting feature number 4282
Extracting feature number 4283
Extracting feature number 4284
Extracting feature number 4285
Extracting feature number 4286
Extracting feature number 4287
Extracting feature number 4288
Extracting feature number 4289
Extracting feature number 4290
Extracting feature number 4291
Extracting feature number 4292
Extracting feature number 4293
Extracting feature number 4294
Extracting feature number 4295
Extracting feature number 4296
Extracting feature number 4297
Extracting feature number 4298
Extracting feature number 4299
Extracting feature number 4300
Extracting feature number 4301
Extracting feature number 4302
Extracting feature number 4303
Extracting feature number 4304
Extracting feature number 4305
Extracting feature number 4306
Extracting feature number 4307
Extracting feature number 4308
Extracti

Extracting feature number 4542
Extracting feature number 4543
Extracting feature number 4544
Extracting feature number 4545
Extracting feature number 4546
Extracting feature number 4547
Extracting feature number 4548
Extracting feature number 4549
Extracting feature number 4550
Extracting feature number 4551
Extracting feature number 4552
Extracting feature number 4553
Extracting feature number 4554
Extracting feature number 4555
Extracting feature number 4556
Extracting feature number 4557
Extracting feature number 4558
Extracting feature number 4559
Extracting feature number 4560
Extracting feature number 4561
Extracting feature number 4562
Extracting feature number 4563
Extracting feature number 4564
Extracting feature number 4565
Extracting feature number 4566
Extracting feature number 4567
Extracting feature number 4568
Extracting feature number 4569
Extracting feature number 4570
Extracting feature number 4571
Extracting feature number 4572
Extracting feature number 4573
Extracti

Extracting feature number 4807
Extracting feature number 4808
Extracting feature number 4809
Extracting feature number 4810
Extracting feature number 4811
Extracting feature number 4812
Extracting feature number 4813
Extracting feature number 4814
Extracting feature number 4815
Extracting feature number 4816
Extracting feature number 4817
Extracting feature number 4818
Extracting feature number 4819
Extracting feature number 4820
Extracting feature number 4821
Extracting feature number 4822
Extracting feature number 4823
Extracting feature number 4824
Extracting feature number 4825
Extracting feature number 4826
Extracting feature number 4827
Extracting feature number 4828
Extracting feature number 4829
Extracting feature number 4830
Extracting feature number 4831
Extracting feature number 4832
Extracting feature number 4833
Extracting feature number 4834
Extracting feature number 4835
Extracting feature number 4836
Extracting feature number 4837
Extracting feature number 4838
Extracti

Extracting feature number 5072
Extracting feature number 5073
Extracting feature number 5074
Extracting feature number 5075
Extracting feature number 5076
Extracting feature number 5077
Extracting feature number 5078
Extracting feature number 5079
Extracting feature number 5080
Extracting feature number 5081
Extracting feature number 5082


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [207]:
# FEATURE EXTRACTION OF THE VALIDATION:
tc_valfeats2 = []
P = 4 #bits of the LBP

for i in range(len(val_labels_tc)):
    print("Extracting feature number", i)
    featim = []
    im = cv2.imread('tc_val_segmented_images/segmented_'+str(i)+'.png')
    #Convert the image to different spaces
    imHSV = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    imLab = cv2.cvtColor(im, cv2.COLOR_BGR2Lab)
    imXYZ = cv2.cvtColor(im, cv2.COLOR_BGR2XYZ)
    
    # COLOR FEATURES:
    for nch in range(3):
        #Extracts each of the channels and removes the zero ones (out of the mask)
        im_ch = im[:,:,nch]
        im_ch = im_ch[im_ch!=0]
        im_ch2 = imHSV[:,:,nch]
        im_ch2 = im_ch2[im_ch2!=0]
        im_ch3 = imLab[:,:,nch]
        im_ch3 = im_ch3[im_ch3!=0]
        im_ch4 = imXYZ[:,:,nch]
        im_ch4 = im_ch4[im_ch4!=0]
        
        #For each channel, computes the mean, std, skew and kurosis
        featim.append(np.mean(im_ch))
        featim.append(np.std(im_ch))
        featim.append(skew(np.ravel(im_ch)))
        featim.append(kurtosis(np.ravel(im_ch)))
        
        featim.append(np.mean(im_ch2))
        featim.append(np.std(im_ch2))
        featim.append(skew(np.ravel(im_ch2)))
        featim.append(kurtosis(np.ravel(im_ch2)))
        
        featim.append(np.mean(im_ch3))
        featim.append(np.std(im_ch3))
        featim.append(skew(np.ravel(im_ch3)))
        featim.append(kurtosis(np.ravel(im_ch3))) 
        
        featim.append(np.mean(im_ch4))
        featim.append(np.std(im_ch4))
        featim.append(skew(np.ravel(im_ch4)))
        featim.append(kurtosis(np.ravel(im_ch4))) 
    
    #TEXTURE FEATURES 
    im_gr = np.array(np.mean(im, axis=2), dtype=np.uint8) #gray image
    glcm = graycomatrix(im_gr, distances = [3] , angles = [0]) #computes the GLCM
    featim.append(graycoprops(glcm, 'contrast')[0,0]) #Computes the features related to GLCM
    featim.append(graycoprops(glcm, 'dissimilarity')[0,0])
    featim.append(graycoprops(glcm, 'homogeneity')[0,0])
    featim.append(graycoprops(glcm, 'energy')[0,0])
    featim.append(graycoprops(glcm, 'correlation')[0,0])
    
    #Computes the LBP
    imLBP = LBP(np.mean(im, axis=2, dtype=np.uint8), P, 2)
    histo_im, _ = np.histogram(imLBP, bins=2**P-1)
    histo_im = histo_im[:-1]
    featim = featim + list(histo_im) #each bin of the histogram is one feature
    featim.append(np.mean(histo_im)) #Adds also the mean and standard deviation of the histogram
    featim.append(np.std(histo_im))
        
    tc_valfeats2.append(featim)

Extracting feature number 0
Extracting feature number 1
Extracting feature number 2
Extracting feature number 3
Extracting feature number 4
Extracting feature number 5
Extracting feature number 6
Extracting feature number 7
Extracting feature number 8
Extracting feature number 9
Extracting feature number 10
Extracting feature number 11
Extracting feature number 12
Extracting feature number 13
Extracting feature number 14
Extracting feature number 15
Extracting feature number 16
Extracting feature number 17
Extracting feature number 18
Extracting feature number 19
Extracting feature number 20
Extracting feature number 21
Extracting feature number 22
Extracting feature number 23
Extracting feature number 24
Extracting feature number 25
Extracting feature number 26
Extracting feature number 27
Extracting feature number 28
Extracting feature number 29
Extracting feature number 30
Extracting feature number 31
Extracting feature number 32
Extracting feature number 33
Extracting feature numbe

Extracting feature number 277
Extracting feature number 278
Extracting feature number 279
Extracting feature number 280
Extracting feature number 281
Extracting feature number 282
Extracting feature number 283
Extracting feature number 284
Extracting feature number 285
Extracting feature number 286
Extracting feature number 287
Extracting feature number 288
Extracting feature number 289
Extracting feature number 290
Extracting feature number 291
Extracting feature number 292
Extracting feature number 293
Extracting feature number 294
Extracting feature number 295
Extracting feature number 296
Extracting feature number 297
Extracting feature number 298
Extracting feature number 299
Extracting feature number 300
Extracting feature number 301
Extracting feature number 302
Extracting feature number 303
Extracting feature number 304
Extracting feature number 305
Extracting feature number 306
Extracting feature number 307
Extracting feature number 308
Extracting feature number 309
Extracting

Extracting feature number 551
Extracting feature number 552
Extracting feature number 553
Extracting feature number 554
Extracting feature number 555
Extracting feature number 556
Extracting feature number 557
Extracting feature number 558
Extracting feature number 559
Extracting feature number 560
Extracting feature number 561
Extracting feature number 562
Extracting feature number 563
Extracting feature number 564
Extracting feature number 565
Extracting feature number 566
Extracting feature number 567
Extracting feature number 568
Extracting feature number 569
Extracting feature number 570
Extracting feature number 571
Extracting feature number 572
Extracting feature number 573
Extracting feature number 574
Extracting feature number 575
Extracting feature number 576
Extracting feature number 577
Extracting feature number 578
Extracting feature number 579
Extracting feature number 580
Extracting feature number 581
Extracting feature number 582
Extracting feature number 583
Extracting

Extracting feature number 825
Extracting feature number 826
Extracting feature number 827
Extracting feature number 828
Extracting feature number 829
Extracting feature number 830
Extracting feature number 831
Extracting feature number 832
Extracting feature number 833
Extracting feature number 834
Extracting feature number 835
Extracting feature number 836
Extracting feature number 837
Extracting feature number 838
Extracting feature number 839
Extracting feature number 840
Extracting feature number 841
Extracting feature number 842
Extracting feature number 843
Extracting feature number 844
Extracting feature number 845
Extracting feature number 846
Extracting feature number 847
Extracting feature number 848
Extracting feature number 849
Extracting feature number 850
Extracting feature number 851
Extracting feature number 852
Extracting feature number 853
Extracting feature number 854
Extracting feature number 855
Extracting feature number 856
Extracting feature number 857
Extracting

Extracting feature number 1095
Extracting feature number 1096
Extracting feature number 1097
Extracting feature number 1098
Extracting feature number 1099
Extracting feature number 1100
Extracting feature number 1101
Extracting feature number 1102
Extracting feature number 1103
Extracting feature number 1104
Extracting feature number 1105
Extracting feature number 1106
Extracting feature number 1107
Extracting feature number 1108
Extracting feature number 1109
Extracting feature number 1110
Extracting feature number 1111
Extracting feature number 1112
Extracting feature number 1113
Extracting feature number 1114
Extracting feature number 1115
Extracting feature number 1116
Extracting feature number 1117
Extracting feature number 1118
Extracting feature number 1119
Extracting feature number 1120
Extracting feature number 1121
Extracting feature number 1122
Extracting feature number 1123
Extracting feature number 1124
Extracting feature number 1125
Extracting feature number 1126
Extracti

In [221]:
#CROPPING OF THE TEST FILES:
# Define a folder to save segmented test images
output_folder_3 = "tc_test_segmented_images"
os.makedirs(output_folder_3, exist_ok=True)

# Load from training data
# Load images and assign labels
test_images_tc, test_labels_tc = load_images_from_folder_with_labels("threeclass/testX", 0)
#bcc_images, bcc_labels = load_images_from_folder_with_labels("threeclass/val/testX", 1)
#scc_images, scc_labels = load_images_from_folder_with_labels("threeclass/val/testX", 2)

#test_images_tc = mel_images + bcc_images + scc_images
#test_labels_tc = mel_labels + bcc_labels + scc_labels

# Lists to store segmented images and features
segmented_images = []
tc_test_features_list2 = []

for i, image in enumerate(test_images_tc):
    print("Loading image... ", i)
    orig_image = image #save the original image for later
    
    # Firstly we detect if there is an halo and crop the image in that case
    preimg = image[:,:,0] #we need only one channel
    pdim = np.shape(preimg)
    
    # These are the corner pixels
    lu = [0,0]
    ru = [0, pdim[1]-1]
    ld = [pdim[0]-1, 0]
    rd = [pdim[0]-1, pdim[1]-1]

    thr = 40 # Intensity threshold under which is considered to be halo
    marg = 30 # Extra cropping margin
    
    #We execute the loop until the four corners exceed the intensity threshold. If they are below, we
    #assume we are still in the halo and keeps iterating
    if ((preimg[0,0] == 0) or (preimg[0, pdim[1]-1]==0)
       or (preimg[pdim[0]-1, 0]==0) or (preimg[pdim[0]-1, pdim[1]-1]==0)):
        while(((preimg[lu[0], lu[1]] < thr) or (preimg[ru[0], ru[1]] < thr)
             or (preimg[ld[0], ld[1]] < thr) or (preimg[rd[0], rd[1]] < thr))
             and ((ru[1]>(2*marg)) and (ld[0]>(2*marg)))): #if iterates too much that exceeds img limits, stops
            lu = [lu[0]+1, lu[1]+1]
            ru = [ru[0]+1, ru[1]-1]
            ld = [ld[0]-1, ld[1]+1]
            rd = [rd[0]-1, rd[1]-1]

        image = image[lu[0]+marg:ld[0]-marg, ru[0]+marg:rd[0]-marg, :]
    
        if ((np.shape(image)[0] < marg) or (np.shape(image)[1]<marg)): #if the cropping is too much, does a more flexible cropping
            image = orig_image[int(0.3*pdim[0]):int(0.7*pdim[1]), int(0.3*pdim[1]):int(0.7*pdim[1])]        
    
    output_filename = os.path.join(output_folder_3, f"segmented_{i}.png")
    segmented_region = image
    cv2.imwrite(output_filename, segmented_region)
    # Extract features using the extract_features function
    features = extract_features(segmented_region)
    tc_test_features_list2.append(features)

Loading image...  0
Loading image...  1
Loading image...  2
Loading image...  3
Loading image...  4
Loading image...  5
Loading image...  6
Loading image...  7
Loading image...  8
Loading image...  9
Loading image...  10
Loading image...  11
Loading image...  12
Loading image...  13
Loading image...  14
Loading image...  15
Loading image...  16
Loading image...  17
Loading image...  18
Loading image...  19
Loading image...  20
Loading image...  21
Loading image...  22
Loading image...  23
Loading image...  24
Loading image...  25
Loading image...  26
Loading image...  27
Loading image...  28
Loading image...  29
Loading image...  30
Loading image...  31
Loading image...  32
Loading image...  33
Loading image...  34
Loading image...  35
Loading image...  36
Loading image...  37
Loading image...  38
Loading image...  39
Loading image...  40
Loading image...  41
Loading image...  42
Loading image...  43
Loading image...  44
Loading image...  45
Loading image...  46
Loading image...  47
Lo

Loading image...  378
Loading image...  379
Loading image...  380
Loading image...  381
Loading image...  382
Loading image...  383
Loading image...  384
Loading image...  385
Loading image...  386
Loading image...  387
Loading image...  388
Loading image...  389
Loading image...  390
Loading image...  391
Loading image...  392
Loading image...  393
Loading image...  394
Loading image...  395
Loading image...  396
Loading image...  397
Loading image...  398
Loading image...  399
Loading image...  400
Loading image...  401
Loading image...  402
Loading image...  403
Loading image...  404
Loading image...  405
Loading image...  406
Loading image...  407
Loading image...  408
Loading image...  409
Loading image...  410
Loading image...  411
Loading image...  412
Loading image...  413
Loading image...  414
Loading image...  415
Loading image...  416
Loading image...  417
Loading image...  418
Loading image...  419
Loading image...  420
Loading image...  421
Loading image...  422
Loading im

Loading image...  751
Loading image...  752
Loading image...  753
Loading image...  754
Loading image...  755
Loading image...  756
Loading image...  757
Loading image...  758
Loading image...  759
Loading image...  760
Loading image...  761
Loading image...  762
Loading image...  763
Loading image...  764
Loading image...  765
Loading image...  766
Loading image...  767
Loading image...  768
Loading image...  769
Loading image...  770
Loading image...  771
Loading image...  772
Loading image...  773
Loading image...  774
Loading image...  775
Loading image...  776
Loading image...  777
Loading image...  778
Loading image...  779
Loading image...  780
Loading image...  781
Loading image...  782
Loading image...  783
Loading image...  784
Loading image...  785
Loading image...  786
Loading image...  787
Loading image...  788
Loading image...  789
Loading image...  790
Loading image...  791
Loading image...  792
Loading image...  793
Loading image...  794
Loading image...  795
Loading im

Loading image...  1119
Loading image...  1120
Loading image...  1121
Loading image...  1122
Loading image...  1123
Loading image...  1124
Loading image...  1125
Loading image...  1126
Loading image...  1127
Loading image...  1128
Loading image...  1129
Loading image...  1130
Loading image...  1131
Loading image...  1132
Loading image...  1133
Loading image...  1134
Loading image...  1135
Loading image...  1136
Loading image...  1137
Loading image...  1138
Loading image...  1139
Loading image...  1140
Loading image...  1141
Loading image...  1142
Loading image...  1143
Loading image...  1144
Loading image...  1145
Loading image...  1146
Loading image...  1147
Loading image...  1148
Loading image...  1149
Loading image...  1150
Loading image...  1151
Loading image...  1152
Loading image...  1153
Loading image...  1154
Loading image...  1155
Loading image...  1156
Loading image...  1157
Loading image...  1158
Loading image...  1159
Loading image...  1160
Loading image...  1161
Loading ima

Loading image...  1476
Loading image...  1477
Loading image...  1478
Loading image...  1479
Loading image...  1480
Loading image...  1481
Loading image...  1482
Loading image...  1483
Loading image...  1484
Loading image...  1485
Loading image...  1486
Loading image...  1487
Loading image...  1488
Loading image...  1489
Loading image...  1490
Loading image...  1491
Loading image...  1492
Loading image...  1493
Loading image...  1494
Loading image...  1495
Loading image...  1496
Loading image...  1497
Loading image...  1498
Loading image...  1499
Loading image...  1500
Loading image...  1501
Loading image...  1502
Loading image...  1503
Loading image...  1504
Loading image...  1505
Loading image...  1506
Loading image...  1507
Loading image...  1508
Loading image...  1509
Loading image...  1510
Loading image...  1511
Loading image...  1512
Loading image...  1513
Loading image...  1514
Loading image...  1515
Loading image...  1516
Loading image...  1517
Loading image...  1518
Loading ima

Loading image...  1833
Loading image...  1834
Loading image...  1835
Loading image...  1836
Loading image...  1837
Loading image...  1838
Loading image...  1839
Loading image...  1840
Loading image...  1841
Loading image...  1842
Loading image...  1843
Loading image...  1844
Loading image...  1845
Loading image...  1846
Loading image...  1847
Loading image...  1848
Loading image...  1849
Loading image...  1850
Loading image...  1851
Loading image...  1852
Loading image...  1853
Loading image...  1854
Loading image...  1855
Loading image...  1856
Loading image...  1857
Loading image...  1858
Loading image...  1859
Loading image...  1860
Loading image...  1861
Loading image...  1862
Loading image...  1863
Loading image...  1864
Loading image...  1865
Loading image...  1866
Loading image...  1867
Loading image...  1868
Loading image...  1869
Loading image...  1870
Loading image...  1871
Loading image...  1872
Loading image...  1873
Loading image...  1874
Loading image...  1875
Loading ima

In [222]:
P = 4
tc_testfeats2 = []

for i in range(len(test_labels_tc)):
    print("Extracting feature number", i)
    featim = []
    im = cv2.imread('tc_test_segmented_images/segmented_'+str(i)+'.png')
    #Convert the image to different spaces
    imHSV = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
    imLab = cv2.cvtColor(im, cv2.COLOR_BGR2Lab)
    imXYZ = cv2.cvtColor(im, cv2.COLOR_BGR2XYZ)
    
    # COLOR FEATURES:
    for nch in range(3):
        #Extracts each of the channels and removes the zero ones (out of the mask)
        im_ch = im[:,:,nch]
        im_ch = im_ch[im_ch!=0]
        im_ch2 = imHSV[:,:,nch]
        im_ch2 = im_ch2[im_ch2!=0]
        im_ch3 = imLab[:,:,nch]
        im_ch3 = im_ch3[im_ch3!=0]
        im_ch4 = imXYZ[:,:,nch]
        im_ch4 = im_ch4[im_ch4!=0]
        
        #For each channel, computes the mean, std, skew and kurosis
        featim.append(np.mean(im_ch))
        featim.append(np.std(im_ch))
        featim.append(skew(np.ravel(im_ch)))
        featim.append(kurtosis(np.ravel(im_ch)))
        
        featim.append(np.mean(im_ch2))
        featim.append(np.std(im_ch2))
        featim.append(skew(np.ravel(im_ch2)))
        featim.append(kurtosis(np.ravel(im_ch2)))
        
        featim.append(np.mean(im_ch3))
        featim.append(np.std(im_ch3))
        featim.append(skew(np.ravel(im_ch3)))
        featim.append(kurtosis(np.ravel(im_ch3))) 
        
        featim.append(np.mean(im_ch4))
        featim.append(np.std(im_ch4))
        featim.append(skew(np.ravel(im_ch4)))
        featim.append(kurtosis(np.ravel(im_ch4))) 
    
    #TEXTURE FEATURES 
    im_gr = np.array(np.mean(im, axis=2), dtype=np.uint8) #gray image
    glcm = graycomatrix(im_gr, distances = [3] , angles = [0]) #computes the GLCM
    featim.append(graycoprops(glcm, 'contrast')[0,0]) #Computes the features related to GLCM
    featim.append(graycoprops(glcm, 'dissimilarity')[0,0])
    featim.append(graycoprops(glcm, 'homogeneity')[0,0])
    featim.append(graycoprops(glcm, 'energy')[0,0])
    featim.append(graycoprops(glcm, 'correlation')[0,0])
    
    #Computes the LBP
    imLBP = LBP(np.mean(im, axis=2, dtype=np.uint8), P, 2)
    histo_im, _ = np.histogram(imLBP, bins=2**P-1)
    histo_im = histo_im[:-1]
    featim = featim + list(histo_im) #each bin of the histogram is one feature
    featim.append(np.mean(histo_im)) #Adds also the mean and standard deviation of the histogram
    featim.append(np.std(histo_im))
        
    tc_testfeats2.append(featim)

Extracting feature number 0
Extracting feature number 1
Extracting feature number 2
Extracting feature number 3
Extracting feature number 4
Extracting feature number 5
Extracting feature number 6
Extracting feature number 7
Extracting feature number 8
Extracting feature number 9
Extracting feature number 10
Extracting feature number 11
Extracting feature number 12
Extracting feature number 13
Extracting feature number 14
Extracting feature number 15
Extracting feature number 16
Extracting feature number 17
Extracting feature number 18
Extracting feature number 19
Extracting feature number 20
Extracting feature number 21
Extracting feature number 22
Extracting feature number 23
Extracting feature number 24
Extracting feature number 25
Extracting feature number 26
Extracting feature number 27
Extracting feature number 28
Extracting feature number 29
Extracting feature number 30
Extracting feature number 31
Extracting feature number 32
Extracting feature number 33
Extracting feature numbe

Extracting feature number 277
Extracting feature number 278
Extracting feature number 279
Extracting feature number 280
Extracting feature number 281
Extracting feature number 282
Extracting feature number 283
Extracting feature number 284
Extracting feature number 285
Extracting feature number 286
Extracting feature number 287
Extracting feature number 288
Extracting feature number 289
Extracting feature number 290
Extracting feature number 291
Extracting feature number 292
Extracting feature number 293
Extracting feature number 294
Extracting feature number 295
Extracting feature number 296
Extracting feature number 297
Extracting feature number 298
Extracting feature number 299
Extracting feature number 300
Extracting feature number 301
Extracting feature number 302
Extracting feature number 303
Extracting feature number 304
Extracting feature number 305
Extracting feature number 306
Extracting feature number 307
Extracting feature number 308
Extracting feature number 309
Extracting

Extracting feature number 551
Extracting feature number 552
Extracting feature number 553
Extracting feature number 554
Extracting feature number 555
Extracting feature number 556
Extracting feature number 557
Extracting feature number 558
Extracting feature number 559
Extracting feature number 560
Extracting feature number 561
Extracting feature number 562
Extracting feature number 563
Extracting feature number 564
Extracting feature number 565
Extracting feature number 566
Extracting feature number 567
Extracting feature number 568
Extracting feature number 569
Extracting feature number 570
Extracting feature number 571
Extracting feature number 572
Extracting feature number 573
Extracting feature number 574
Extracting feature number 575
Extracting feature number 576
Extracting feature number 577
Extracting feature number 578
Extracting feature number 579
Extracting feature number 580
Extracting feature number 581
Extracting feature number 582
Extracting feature number 583
Extracting

Extracting feature number 825
Extracting feature number 826
Extracting feature number 827
Extracting feature number 828
Extracting feature number 829
Extracting feature number 830
Extracting feature number 831
Extracting feature number 832
Extracting feature number 833
Extracting feature number 834
Extracting feature number 835
Extracting feature number 836
Extracting feature number 837
Extracting feature number 838
Extracting feature number 839
Extracting feature number 840
Extracting feature number 841
Extracting feature number 842
Extracting feature number 843
Extracting feature number 844
Extracting feature number 845
Extracting feature number 846
Extracting feature number 847
Extracting feature number 848
Extracting feature number 849
Extracting feature number 850
Extracting feature number 851
Extracting feature number 852
Extracting feature number 853
Extracting feature number 854
Extracting feature number 855
Extracting feature number 856
Extracting feature number 857
Extracting

Extracting feature number 1095
Extracting feature number 1096
Extracting feature number 1097
Extracting feature number 1098
Extracting feature number 1099
Extracting feature number 1100
Extracting feature number 1101
Extracting feature number 1102
Extracting feature number 1103
Extracting feature number 1104
Extracting feature number 1105
Extracting feature number 1106
Extracting feature number 1107
Extracting feature number 1108
Extracting feature number 1109
Extracting feature number 1110
Extracting feature number 1111
Extracting feature number 1112
Extracting feature number 1113
Extracting feature number 1114
Extracting feature number 1115
Extracting feature number 1116
Extracting feature number 1117
Extracting feature number 1118
Extracting feature number 1119
Extracting feature number 1120
Extracting feature number 1121
Extracting feature number 1122
Extracting feature number 1123
Extracting feature number 1124
Extracting feature number 1125
Extracting feature number 1126
Extracti

Extracting feature number 1360
Extracting feature number 1361
Extracting feature number 1362
Extracting feature number 1363
Extracting feature number 1364
Extracting feature number 1365
Extracting feature number 1366
Extracting feature number 1367
Extracting feature number 1368
Extracting feature number 1369
Extracting feature number 1370
Extracting feature number 1371
Extracting feature number 1372
Extracting feature number 1373
Extracting feature number 1374
Extracting feature number 1375
Extracting feature number 1376
Extracting feature number 1377
Extracting feature number 1378
Extracting feature number 1379
Extracting feature number 1380
Extracting feature number 1381
Extracting feature number 1382
Extracting feature number 1383
Extracting feature number 1384
Extracting feature number 1385
Extracting feature number 1386
Extracting feature number 1387
Extracting feature number 1388
Extracting feature number 1389
Extracting feature number 1390
Extracting feature number 1391
Extracti

Extracting feature number 1625
Extracting feature number 1626
Extracting feature number 1627
Extracting feature number 1628
Extracting feature number 1629
Extracting feature number 1630
Extracting feature number 1631
Extracting feature number 1632
Extracting feature number 1633
Extracting feature number 1634
Extracting feature number 1635
Extracting feature number 1636
Extracting feature number 1637
Extracting feature number 1638
Extracting feature number 1639
Extracting feature number 1640
Extracting feature number 1641
Extracting feature number 1642
Extracting feature number 1643
Extracting feature number 1644
Extracting feature number 1645
Extracting feature number 1646
Extracting feature number 1647
Extracting feature number 1648
Extracting feature number 1649
Extracting feature number 1650
Extracting feature number 1651
Extracting feature number 1652
Extracting feature number 1653
Extracting feature number 1654
Extracting feature number 1655
Extracting feature number 1656
Extracti

Extracting feature number 1890
Extracting feature number 1891
Extracting feature number 1892
Extracting feature number 1893
Extracting feature number 1894
Extracting feature number 1895
Extracting feature number 1896
Extracting feature number 1897
Extracting feature number 1898
Extracting feature number 1899
Extracting feature number 1900
Extracting feature number 1901
Extracting feature number 1902
Extracting feature number 1903
Extracting feature number 1904
Extracting feature number 1905
Extracting feature number 1906
Extracting feature number 1907
Extracting feature number 1908
Extracting feature number 1909
Extracting feature number 1910
Extracting feature number 1911
Extracting feature number 1912
Extracting feature number 1913
Extracting feature number 1914
Extracting feature number 1915
Extracting feature number 1916
Extracting feature number 1917
Extracting feature number 1918
Extracting feature number 1919
Extracting feature number 1920
Extracting feature number 1921
Extracti

In [223]:
#We firstly join both sets of features in the train and val sets
tc_alltrain = np.concatenate([np.array(tc_train_features_list2), np.array(tc_trainfeats2)], axis=1)
tc_allval = np.concatenate([np.array(tc_val_features_list2), np.array(tc_valfeats2)], axis=1)
print(np.shape(tc_alltrain), np.shape(tc_allval))
#And now we join the train and val sets, to have more data
tc_alltrainval = np.concatenate([tc_alltrain, tc_allval], axis = 0)
print(np.shape(tc_alltrainval))
train_labels_tc = [0 for x in range(2713)] + [1 for x in range(1993)] + [2 for x in range(376)] 
all_labels_tc = train_labels_tc + val_labels_tc #we have to join the labels as well
print(np.shape(all_labels_tc))

# and we join features for the test set as well
tc_test = np.concatenate([np.array(tc_test_features_list2), np.array(tc_testfeats2)], axis=1)
print(np.shape(tc_test))

(5082, 84) (1270, 84)
(6352, 84)
(6352,)
(2121, 84)


In [227]:
#Now we will do a double classification. First we classify between melanoma and no-melanoma, 
#so we convert the 2 in the label vector to 1 as well.
bin1_train_labels = np.zeros(len(all_labels_tc), dtype=np.uint8)
bin1_train_labels[np.array(all_labels_tc)!=0] = 1 #both bcc and scc are non-melanoma

X_train = np.array(tc_alltrainval) #BOTH
X_train[np.isnan(X_train)] = 0 #we remove any possible NaN
X_test = np.array(tc_test)

y_train = np.array(bin1_train_labels)

#We scale the data before classifying
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#We use random forest for this first classification
rf_classifier = RFC(n_estimators=100, random_state=3)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test) #predicted values are 0 (melanoma) and 1 (non-melanoma)

#Now we do a second classification between bcc and scc, so to train we use only the non-melanoma samples
bin2_train_labels = np.array(all_labels_tc)[np.array(all_labels_tc)!=0]

X_train = np.array(tc_alltrainval)[np.array(all_labels_tc)!=0, :]
X_train[np.isnan(X_train)] = 0 #we remove possible NaNs
X_test = np.array(tc_test)[y_pred!=0,:]

#As this second classification is unbalanced, we combine undersampling and oversampling to balance data
X_train, bin2_train_labels = RUS(random_state=40, sampling_strategy = 0.25).fit_resample(X_train, bin2_train_labels)
X_train, bin2_train_labels = SMOTE(random_state=42, sampling_strategy=0.50).fit_resample(X_train, bin2_train_labels)

y_train = np.array(bin2_train_labels)

#We scale data again
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#And we use again RF, but now using the class_weights as well to balance a bit more the data
rf_classifier2 = RFC(n_estimators=100, random_state=2, class_weight = {1: 1.2, 2: 1})
rf_classifier2.fit(X_train, y_train)
y_pred2 = rf_classifier2.predict(X_test)

y_pred_final = np.zeros(len(y_pred), dtype=np.uint8)
y_pred_final[y_pred!=0] = y_pred2 #we add the predicted values to the final prediction vector, to the corresponding positions


In [230]:
import csv

#Finally, we write the csv file.
with open('threeclass.csv', 'w', newline='') as csvfile:
    for lab in y_pred_final:
        lab_writer = csv.writer(csvfile)
        lab_writer.writerow(str(lab))